# Develop nested cross validation strategy for model training

In [9]:
# Load an example dataset
response_path = "/mnt/c/Users/nick/Desktop/courses/rotations/corsello_lab/data/responsedir/amg-232_v2.csv"
feature_dir = "/mnt/c/Users/nick/Desktop/courses/rotations/corsello_lab/data/depmap_public-22q1-305b_v24"
output_dir = "test_output"
config_path = ""


In [10]:
import os
import sys
sys.path.append("../refract")

In [43]:
import os
from datasets import ResponseSet, FeatureSet
from trainers import RFBaseTrainer
from utils import RandomForestConfig
import json
import argparse
import logging
from sklearn.model_selection import KFold
import pandas as pd

In [12]:
response_set = ResponseSet(response_path)
response_set.load_response_table()

In [13]:
feature_set = FeatureSet(feature_dir)

In [14]:
feature_set.load_concatenated_feature_tables()

In [15]:
config = RandomForestConfig()

In [16]:
feature_set.feature_tables["all"]

,CNA_TEX47,MUT_other_OR4N2,GE_KCNQ1,GE_KBTBD8,MUT_dam_KIF5C,MUT_other_KDM2A,XPR_PPARGC1A,MUT_other_VPS51,MUT_dam_OR51M1,MUT_other_MKX,...,GE_PCSK9,CNA_ATP6V0A1,MUT_hs_ECHDC2,XPR_SIDT2,GE_CCN6,MUT_other_NUDT16,CNA_ENOX2,CNA_H2AC4,MUT_other_DDHD1,GE_HSP90AA1
ccle_name,,,,,,,,,,,,,,,,,,,,,
DEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,1.164438,0,0.214125,2.386811,0,0,0.099843,0,0,0,...,0.042644,0.968154,0,-0.047746,0.201634,0,0.697453,1.295837,0,9.236134
SNU1196_BILIARY_TRACT,1.068606,0,3.063503,0.722466,0,0,-0.030555,0,0,0,...,3.372952,1.227187,0,0.087754,0.070389,0,0.690040,0.882281,0,10.070389
ABC1_LUNG,0.958219,0,0.014355,0.555816,0,0,0.126369,0,0,0,...,4.068671,0.914720,0,0.069003,0.124328,0,0.990727,1.049125,0,9.494336
KE97_STOMACH,0.999870,0,1.510962,2.260026,0,0,0.183388,0,0,0,...,0.124328,1.017689,0,0.242637,0.176323,0,0.528975,0.991640,0,10.110744
YKG1_CENTRAL_NERVOUS_SYSTEM,1.352160,0,0.084064,1.275007,0,0,-0.131423,0,0,0,...,0.111031,1.169646,0,-0.072516,0.084064,0,0.963582,1.141589,0,10.539934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCIH2023_LUNG,1.061771,0,0.378512,0.584963,0,0,0.079166,0,0,0,...,4.333424,1.077782,0,0.027784,0.056584,0,0.432978,0.686634,0,9.824338
NB4_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,1.265541,0,0.097611,1.510962,0,0,0.146733,0,0,0,...,0.028569,0.937935,0,-0.140093,0.111031,0,0.658764,1.089041,0,9.887876
COLO792_SKIN,1.401898,0,0.028569,0.056584,0,0,0.174617,0,0,0,...,0.028569,1.403473,0,-0.025223,0.000000,0,0.779200,1.423141,0,7.509221


In [22]:
# DEV: filter the feature table to get a reduced set of features.
feature_cols = feature_set.feature_tables["all"].columns
# randomly select 50 columns
feature_cols = feature_cols[0:50]
feature_set.feature_tables["all"] = feature_set.feature_tables["all"].loc[:, feature_cols]

In [36]:
df = response_set.LFC
pert_name = df.pert_name.unique()[0]
pert_mfc_id = df.pert_mfc_id.unique()[0]
dose = df.dose.unique()[0]
X, y = response_set.get_joined_features(
    pert_name=pert_name,
    pert_mfc_id=pert_mfc_id,
    dose=dose,
    feature_set=feature_set,
    feature_name="all",
)

In [37]:
"""
p_grid = {
    "n_estimators": [50, 100, 200, 300],
    "max_depth": [None, 5, 10, 15],
    "min_samples_split": [2, 5, 10],
}
"""
p_grid = {
    "n_estimators": [50, 100],
    "max_depth": [None, 5],
}

In [67]:
# k fold split
# k is 5
kf_outer = KFold(n_splits=5, shuffle=True, random_state=42)

In [68]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score

In [69]:
from sklearn.metrics import mean_squared_error, make_scorer

mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [70]:
cv_results = []

for idx, (train_val_index, test_index) in enumerate(kf_outer.split(X,y)):
    X_train_val, X_test = X.iloc[train_val_index], X.iloc[test_index]
    y_train_val, y_test = y.iloc[train_val_index], y.iloc[test_index]
    y_train_val = y_train_val.values.ravel()
    y_tesst = y_test.values.ravel()

    # grid search
    grid = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid=p_grid,
        cv=4,
        scoring=mse_scorer,
        n_jobs=-1,
        verbose=1,
        refit=True,
    )

    # fit the grid search model
    grid.fit(X_train_val, y_train_val)
    # get the best estimator 
    best_model = grid.best_estimator_
    # get the best parameters
    best_params = grid.best_params_

    # get the train_val score
    train_val_mse = mean_squared_error(best_model.predict(X_train_val), y_train_val)
    # get the test score
    test_mse = mean_squared_error(best_model.predict(X_test), y_test)

    # save the results
    fold_out = {}
    fold_out["train_val_index"] = train_val_index
    fold_out["test_index"] = test_index
    fold_out["best_params"] = best_params
    fold_out["train_mse"] = train_val_mse
    fold_out["test_mse"] = test_mse
    fold_out["best_model"] = best_model
    cv_results.append(fold_out)


Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits


In [71]:
cv_results = pd.DataFrame(cv_results)

In [72]:
cv_results

,train_val_index,test_index,best_params,train_mse,test_mse,best_model
0,"[1, 2, 4, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17,...","[0, 3, 5, 9, 15, 22, 25, 30, 31, 33, 39, 42, 4...","{'max_depth': None, 'n_estimators': 100}",0.165060,1.093691,"(DecisionTreeRegressor(max_features=1.0, rando..."
1,"[0, 1, 2, 3, 4, 5, 8, 9, 11, 12, 13, 14, 15, 2...","[6, 7, 10, 16, 17, 18, 19, 24, 36, 59, 60, 66,...","{'max_depth': None, 'n_estimators': 50}",0.192926,0.818212,"(DecisionTreeRegressor(max_features=1.0, rando..."
2,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[2, 11, 23, 26, 29, 37, 38, 67, 68, 69, 74, 83...","{'max_depth': None, 'n_estimators': 100}",0.170263,1.211365,"(DecisionTreeRegressor(max_features=1.0, rando..."
3,"[0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 15, 16, 1...","[4, 8, 12, 14, 27, 28, 32, 35, 40, 41, 44, 47,...","{'max_depth': None, 'n_estimators': 100}",0.167556,1.155590,"(DecisionTreeRegressor(max_features=1.0, rando..."
4,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15...","[1, 13, 20, 21, 34, 43, 48, 49, 50, 52, 53, 54...","{'max_depth': 5, 'n_estimators': 100}",0.457469,1.759164,"(DecisionTreeRegressor(max_depth=5, max_featur..."


Train/Val: 303 Test: 76


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
